_____

<center><h1>Introduction to Data Science</h1></center>

<center><a target="_blank" href="https://learning.constructor.org/"><img src="https://drive.google.com/uc?id=1wxkbM60NlBlkbGK1JqUypKL24RrTiiYk" width="200" style="background:none; border:none; box-shadow:none;" /></a> </center>

<p style="margin-bottom:1cm;"></p>

_____

<center> <h2> Workshop  </h2> </center>

<p style="margin-bottom:1cm;"></p>

_____

<center>Constructor Learning, 2023</center>

_____


Welcome to this Data Analytics workshop! We hope you enjoy it, but most importantly have fun.

Here's what we will cover today:

- Data Science Lifecycle
- Data Wrangling
- Exploratory Data Analysis
- Intro to Machine Learning

---
First let's talk about the lifecycle of a Data Science project.



<a target="_blank" href="https://learn.microsoft.com/en-us/azure/architecture/data-science-process/lifecycle"><img src="https://learn.microsoft.com/en-us/azure/architecture/data-science-process/media/lifecycle/tdsp-lifecycle2.png" width="800" style="background:none; border:none; box-shadow:none;" /></a>

<center><h1>Live Coding</h1></center>

This is a copy notebook (file) of the original version. For your progress to be saved, you can go to `File` and then `Save a copy in Drive`. This is optional

---
### How are we going to start our workshop fun?? 🐍

The most popular programming language used in Data Science is [Python](https://www.python.org/). It is one of the most accessible yet very powerful programming languages available. It has a simplified syntax, which gives emphasis on natural language, making it easier to learn and read.

But a programming languages needs a place to be written and executed. Just like we need Google Docs to write a novel, we need a [Colab Notebook](https://research.google.com/colaboratory/faq.html) (Or Jupyter Notebook) to write our amazing code. That is precisely what you are using right now on your browser, a computational web session that allows you to input and output python code.

It makes the job of exploring data a lot more fun! Let's get started!

# Load Dependencies

---
Python code is very powerful, but there are many operations that can get tedious if we were to code them ourselves. 

That is why several amazing authors all over the world have created libraries that help us perform specific tasks more efficiently and with less lines of code.

For this workshop we will use the following libraries:
- **Pandas**: the most important tool of a Data Scientist. It offers many tools for data manipulation and analysis.
- **Plotly**: an interacting graphing library that allows us to plot the data and create visuals.
- **Scikit-Learn**: Simple and efficient tools for predictive data analysis Accessible to everybody, and reusable in various contexts.

In [ ]:
# Some libraries do not come preinstalled in Google Colab, so we need to manually install 
# them using the next line of code.
!pip install pgeocode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pgeocode

import plotly.express as px
import plotly.graph_objects as go

# Data Wrangling

## Load Dataset

---
Data can come from many different sources, and depending on the format it is in, the way to import it will vary. In the case of the "Swiss Housing Prices dataset"*, the data is stored in Google Drive as a `.csv` file. The next block of code will load the data to the notebook into a Pandas DataFrame object.

\*Data scraped by [Ansam Zedan](https://www.linkedin.com/in/ansam-zedan/) on the Homegate.ch website.

In [ ]:
# Read the home price csv file from the URL
orig_url = "https://drive.google.com/file/d/14a0l9bT9DLFqIu5rtyiTc9hLC555rvOz/view?usp=sharing"
file_id = orig_url.split('/')[-2]
data_path='https://drive.google.com/uc?export=download&id=' + file_id

df = pd.read_csv(data_path, index_col=0)
df

---
The first method we will use on our newly created dataframe is `.info()`. Here we can get a first overview of the types of data we have, and if we have missing values.

In [ ]:
df.info()

---
The price column is formatted with alphanumeric values. In order to properly do data exploration, we need to treat this column as an integer (number), so let's clean this entries using a Regular Expression (regex) so it only keeps the digits

In [ ]:
df["price"]

In [ ]:
df["price"].sample(n=10).unique()

In [ ]:
df['price'] = df['price'].str.replace('[^0-9]', '')
df.head()

---
Now we have only numeric values for the price column, but will it be treated as an integer, or will Pandas still interpret it as a string?

In [ ]:
df.info()

---
Even though we stripped the values of non-numerical characters, we still need to convert the data type so it can be interpreted as an integer. This way we can later take advantage of this for plotting and applying methods if needed.

In [ ]:
df['price'] = df['price'].astype(int)
df.info()

---
Another column which should be numerical but it's being treated as an object (string) is the "area_m2" column. In this case it would be beneficial to also strip the "m2" out of the values and only keep the numbers.



In [ ]:
df.head(3)

In [ ]:
df['area_m2'] = df['area_m2'].str.split(' ').str[0]
df['area_m2'] = df['area_m2'].astype(float)
df.head(3)

---
When working with data, many times there will be missing values in some of the samples. This is a normal situation to encounter when analyzing the data and emphasizes the importance of knowing the data. 

There are several approaches to deal with missing values:
- Substitute the missing values with the mean, median, mode or arbitrary value.
- Drop the samples with missing data.
- Impute the missing values using Machine Learning.

---
The column "floors_num" indicates the number of floors that the property has. There are many entries with missing values in this column. We can inspect the data to make a more educated inference of which value to assign to the rows with missing data.

We can do this by visualizing the counts of each type of property and seeing that most of the properties with missing values are apartments, which most of the time only have one floor.

In [ ]:
print(f"Missing values in 'floors_num': {df['floors_num'].isna().sum()}")

In [ ]:
def get_type_proportion(request):
    if request == "Is Nan":
        conditional = df['floors_num'].isna()
    elif request == "1":
        conditional = df['floors_num'] == 1
    elif request == "More than 1":
        conditional = df['floors_num'] > 1

    s1 = df[conditional]['type'].value_counts()[0:5].sort_index()
    s2 = pd.Series([df[conditional]['type'].value_counts()[5:].sum()], index=["Other"])
    return pd.concat([s1, s2], ignore_index=False)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(22,5))

inspect_df = get_type_proportion("1")
ax[0].pie(inspect_df, labels=inspect_df.index, autopct='%1.f%%', startangle=90)
ax[0].set_title("Properties With One Floor", fontsize=18)

inspect_df = get_type_proportion("More than 1")
ax[1].pie(inspect_df, labels=inspect_df.index, autopct='%1.f%%', startangle=90)
ax[1].set_title("Properties With More Than One Floor", fontsize=18)

inspect_df = get_type_proportion("Is Nan")
ax[2].pie(inspect_df, labels=inspect_df.index, autopct='%1.f%%', startangle=90)
ax[2].set_title("Properties With Missing Floor Number", fontsize=18)

fig.show();

---
Looking at the distribution of the types of properties according to the amount of floor levels they have, we can make the following assumption: properties with one floor level are mostly Apartments (73%), so since the majority of properties missing this value are also Apartments (49%), it's more likely that they also have one floor level.

In [ ]:
df['floors_num'] = df['floors_num'].fillna(1).astype(int)
df.head()

---
The "floor" column which represents the level the property is located at also has several missing values. We will infer that no given data means it is a Ground Floor property.

In [ ]:
print(f"Missing values in 'floor': {df['floor'].isna().sum()}")

In [ ]:
# EXERCISE 1
# Fill the missing values of the 'floor' column with "GF"
         # <-- Your code here
df.tail()

---
For the "last_refurbishment" column, which states the year of the last remodeling, there are also missing values. We will infer that this is because the building has not been refurbished since its construction, and it makes sense to set the date as the year which it was built.

In [ ]:
print(f"Missing values in 'last_refurbishment': {df['last_refurbishment'].isna().sum()}")

In [ ]:
df["year_built"]

In [ ]:
# EXERCISE 2
# Fill the missing values in the column 'last_refurbishment' with the values from the column 'year_built'
       # <--- Your code here
df

## Basic feature extraction

--- 
From data we already have, we can create new features. For example, using the area of the property and the price, we can also get the price per square meter.

In [ ]:
df = df[df["area_m2"].notna()].copy()
df["price_sqm"] = df['price'] / df['area_m2']
df["price_sqm"] = df["price_sqm"].astype(int)
print(df.shape)
df.head()

---
We can get a few more features from the address column: like separating the zip code and the city, and even getting the coordinates for each property.

In [ ]:
for address in df['address'].sample(n=20).unique():
  print(address)

---
We see that some addresses only contain the zip code and city, but others also contain the full address. Let's extract these values.

In [ ]:
def extract_zip_city(address):
    if ',' in address:
        zip_and_city = address.split(', ')[1]
        zip_code = zip_and_city.split(' ')[0]
        city = zip_and_city.split(' ')[1]
    else:
        zip_and_city = address
        zip_code = zip_and_city.split(' ')[0]
        city = zip_and_city.split(' ')[1]
    return pd.Series([zip_code, city])

df[['zip_code', 'city']] = df['address'].apply(extract_zip_city)
df

---
Using the library `pgeocode`, which we installed at the beginning of the notebook, we can use the zip code number to get the name of the canton and the coordinates we will later use.

In [ ]:
pgeocode_nomi = pgeocode.Nominatim('ch')
pgeocode_nomi.query_postal_code("8134")

In [ ]:
def add_canton(zip_code):
    zip_info = pgeocode_nomi.query_postal_code(zip_code)
    return zip_info["state_name"]

df["canton"] = df["zip_code"].apply(add_canton)
df.head(3)

In [ ]:
# EXERCISE 3
def add_coordinates(zip_code): 
    zip_info = pgeocode_nomi.query_postal_code(zip_code)
    # Assign two variables called 'latitude' and 'longitude' with the corresponding keys from the 'zip_info' data
    latitude =  #<-- Your code here
    longitude =  #<-- Your code here
    return pd.Series([latitude, longitude])

df[["lat", "lon"]] = df["zip_code"].apply(add_coordinates)
df.head(3)

--- 
We no longer need the full address, so we can drop this column.

In [ ]:
df = df.drop('address', axis=1)

---
Now we have our final data frame which we will use to do analysis, gather insights and create a machine learning model!

In [ ]:
df

# Basic EDA (Exploratory Data Analysis)

## Data Distribution 

<img src="https://pbs.twimg.com/media/E5ePcUdVkAEvEX6?format=jpg&name=small" width="250" style="background:none; border:none; box-shadow:none;" />

<img src="https://upload.wikimedia.org/wikipedia/commons/c/cc/Relationship_between_mean_and_median_under_different_skewness.png" width="700" style="background:none; border:none; box-shadow:none;" />



In [ ]:
fig, ax = plt.subplots(1, 2,figsize=(20,5))

sns.histplot(data=df, x='price', kde=True, stat='density', ax=ax[0])
sns.histplot(data=df, x='area_m2', kde=True, stat='density', ax=ax[1])

fig.suptitle('Density of Price and Area', fontsize=18)
fig.show();

In [ ]:
fig = px.histogram(df, x="price",
                   marginal="box",
                   hover_data=df.columns)
fig.update_layout(
    font={"size":17},
    title_text="Price Distribution on Histogram and Boxplot", 
    title_x=0.5,
    )

fig.show()

---
**Understanding Box Plots**

<center><img src="https://miro.medium.com/max/9000/1*2c21SkzJMf3frPXPAR_gZA.png" width="700" style="background:none; border:none; box-shadow:none;" /></center>

In [ ]:
fig = px.box(df, x="type", y="price") 
fig.update_layout(
    font={"size":17},
    title_text="Boxplot Distribution Between Property Types", 
    title_x=0.5,
    )
fig.update_xaxes(tickangle=-45)
fig.show()

## Scatter Plot

In [ ]:
fig = px.scatter(df, x="area_m2", y="price", color='type')

fig.update_layout(
    font={"size":17},
    title_text="Correlation Between Property Area And Its Price", 
    title_x=0.5,
)

fig.show()

In [ ]:
# EXERCISE 4
# Plot a scatter plot with the correlation between price and number of roooms

fig =  #<-- Your code here

fig.update_layout(
    font={"size":17},
    title_text="Correlation Between Property Number Of Rooms And Its Price", 
    title_x=0.5,
)

fig.show()

## Viewing Variable Correlations in a Heatmap

In [ ]:
corr_matrix = df.corr()
corr_matrix

In [ ]:
def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)

trimask = np.triu(np.ones_like(corr_matrix, dtype=bool))


fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = corr_matrix.columns,
        y = corr_matrix.index,
        z = np.array(corr_matrix),
        text=trunc(np.array(corr_matrix), decs=2), texttemplate="%{text}",
        colorscale = 'RdBu', ygap=1, xgap=1
    )
)

fig.update_layout(
    title_text="Correlation Heatmap", 
    title_x=0.5,
    width=1000, 
    height=600,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    yaxis_autorange='reversed'
)

fig.show()

# More Visualizations

## Mapping

In [ ]:
fig = px.scatter_mapbox(
    df, 
    lat="lat", 
    lon="lon", 
    hover_name="price", 
    color="canton", 
    size="price",
    zoom=7, 
    center={"lat":46.8182, "lon":8.2275}
)

fig.update_layout(
    mapbox_style="carto-positron", 
    margin={"r":0,"t":0,"l":0,"b":0}, 
    height=600, 
    font={"size":17}
)

fig.show()

In [ ]:
# EXERCISE 5
df_map = df[df['price']<5000000]

fig = px.scatter_mapbox(
    df_map, 
    lat="lat", 
    lon="lon", 
    hover_name="price", 
    color=???,   #<--- Make the map show the color scale from the price values
    zoom=7, 
    center={"lat":46.8182, "lon":8.2275},
)

fig.update_layout(
    mapbox_style="carto-positron", 
    margin={"r":0,"t":0,"l":0,"b":0},
    height=600, 
    font={"size":17}
)

fig.show()

# Intro to Machine Learning

## Further Data Cleaning

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_title('Distribution Of Price')
ax.boxplot(df['price'])
fig.show();

In [ ]:
df = df[df['price']< 5000000].reset_index(drop=True)
#df = df[~df["year_built"].isna()]
df.drop(['lat', 'lon', 'price_sqm'], axis=1, inplace=True, errors='ignore')
df

---
Machine learning is the process of teaching a computer to learn patterns from data and then to apply those patterns to make preditions on new data. In traditional programming, you write rules to tell the computer exactly what to do. For example, if you want to write a program that converts miles to kilometers, you would write a function that computes the following equation:

<span>
<img src="https://drive.google.com/uc?id=1aa50Dd83JwO7x_SOWbNckj2ThBOfVdeb" width="40%"/>


But in ML, instead of writing the rule, you provide the computer a lot of examples of input data as well as the desired output, say many samples miles to kilometer conversion data. Then let the computer learn the rule itself.


<img src="https://drive.google.com/uc?id=1sj2IeZGi9RI6VH-ZvFpFdS2fC6e3XO0R" width="40%"/>
</span>

But there are many cases where the rules are not that simple. For example, this very dataset of housing prices takes into consideration many variables, and it would be very complicated to write a formula ourselves.

ML is ideal for these types of problems, where you have lots of data that have complex relationships that would be very difficult for humans to manually create rules for.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn import metrics
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor

---
Don't worry too much about the following code. There are several technicalities better saved for another time. The only important thing to understand is that we separate our data from the independent (x) and dependent variables (y), since we want the ML algorithm to learn the patterns from the independent variables that give the target output.

In [ ]:
X = df.drop(columns=['price'])
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
numeric_features = X_train.select_dtypes(exclude=['object']).columns.tolist()
categorical_transformer = Pipeline(steps=[
                                          ("onehot", OneHotEncoder(handle_unknown="ignore"))
                                          ])
numeric_transformer = Pipeline(steps=[
                                      ("knn_imputer", KNNImputer(n_neighbors=5)),
                                      ("scaler", MinMaxScaler())
                                      ])
preprocessor = ColumnTransformer(transformers=[
                                               ("num", numeric_transformer, numeric_features),
                                               ("cat", categorical_transformer, categorical_features)
                                               ])

model = RandomForestRegressor(n_estimators=1000)

pipeline_model = Pipeline(steps=[
                              ("pre_process", preprocessor), 
                              ("model", model)
                              ])

pipeline_model.fit(X_train, y_train)

In [ ]:
pred = pipeline_model.predict(X_test)

print('MAE', metrics.mean_absolute_error(y_test, pred))
print('R2 Score', metrics.r2_score(y_test, pred))

---
Enter the values of a property you would like to predict its price for:

In [ ]:
# EXERCISE 6

target_property = {
    'type' : ['Apartment'],
    'room_num' : [2.5],
    'floor' : ["2"],
    'area_m2' : [80],
    'floors_num' : [1],
    'year_built' : [1990],
    'last_refurbishment' : [2002],
    'zip_code' : ["8003"],
    'city' : ["Zürich"],
    'canton' : ["Kanton Zürich"],
}

to_predict = pd.DataFrame(target_property)
to_predict[['area_m2', 'year_built', 'last_refurbishment']] = to_predict[['area_m2', 'year_built', 'last_refurbishment']].astype(float)
to_predict

In [ ]:
pred = pipeline_model.predict(to_predict)
print(f"The value of the property using the trained machine learning algorithm is of {round(pred[0])} CHF")

---
Congratulations! 

- You learned how to start a Data Science project.
- You learned how to do data wrangling to clean the data.
- You learned how to do exploratory data analysis and visualize insights with plots.
- And you trained a Machine Learning Algorithm that allows user to get a predicted price of a property based on previous data.

It feels awesome to know all these tools.

From the Constructor Learning team, we thank you for your participation!


## Solutions to the exercises:


Exercise 1: 
```
df['floor'] = df['floor'].fillna("GF")
```

Exercise 2:
```
df['last_refurbishment'] = df['last_refurbishment'].fillna(df['year_built'])
```

Exercise 3:
```
latitude = zip_info["latitude"]  #<-- Make this EXERCISE
longitude = zip_info["longitude"] #<-- Make this EXERCISE
```

Exercise 4:
```
fig = px.scatter(df, x="room_num", y="price", color='type')
```

Exercise 5:
```
fig = px.scatter_mapbox(
    df_map, 
    lat="lat", 
    lon="lon", 
    hover_name="price", 
    color="price", 
    zoom=7, 
    center={"lat":46.8182, "lon":8.2275},
)
```